In [0]:
import os

dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]
for npz_file in npz_files:
    npz_file_path = dataset_dir + "/" + npz_file
    print(f"npz_file_path : {npz_file_path}")

In [9]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-381-F10-200.npz
Computational Resource: cpu
tensor([[10.7732, 11.7255, 11.1167,  ...,  0.3064,  0.6288,  1.5719],
        [10.9542,  4.5473, -4.8443,  ...,  1.7207,  2.8269,  4.3059],
        [-0.5828, -6.5629, -4.3325,  ...,  7.1175,  6.8424,  6.3382],
        ...,
        [ 0.2805, -0.1437,  1.3464,  ..., 14.6067, 15.3504, 15.0539],
        [-0.4749,  0.2884,  1.6568,  ...,  3.2846,  3.5197, -1.5238],
        [ 1.1973,  1.8182,  3.3854,  ...,  5.0264, -0.1825, -0.5718]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7725, 0.3963, 0.3713, 0.3541, 0.3599, 0.2867, 0.2922, 0.3430, 0.4005,
         0.5784],

In [8]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F10-500.npz
Computational Resource: cpu
tensor([[-0.4719,  2.3044,  3.4747,  ...,  2.6430,  2.8964,  3.4605],
        [ 2.2450,  4.1074,  7.1179,  ..., 11.7417, 10.5587,  8.7491],
        [ 9.9247,  8.9686,  2.9052,  ..., -0.8356, -7.6320, -4.2512],
        ...,
        [ 3.4964,  2.5382,  2.4924,  ...,  4.0678,  2.2567,  0.4528],
        [ 8.8259,  8.6446,  8.5316,  ...,  5.0953,  2.9231,  2.3225],
        [-2.1671, -3.1036, -1.1332,  ..., -1.2870, -0.5012,  0.7894]])
shape: torch.Size([10, 500])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7570, 0.3932, 0.4183, 0.3566, 0.3180, 0.2369, 0.2713, 0.3419, 0.4302,
         0.5967],

In [11]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F10/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F10-1000.npz
Computational Resource: cpu
tensor([[ 2.2037,  3.6098, -0.2965,  ..., -2.3251, -1.2428, -0.5167],
        [-1.8406,  0.0763,  1.1999,  ..., -6.6357, -5.0896, -3.5993],
        [ 1.8289,  2.7404,  3.7013,  ...,  2.2962, -2.3941, -3.9358],
        ...,
        [ 3.5459,  0.8456, -0.6586,  ...,  1.8071, -0.6074, -2.8640],
        [13.8677, 14.0580, 13.4736,  ...,  0.8452,  0.9258,  3.6444],
        [-0.8433,  1.0665,  5.0794,  ..., -0.4879,  0.1999,  1.3539]])
shape: torch.Size([10, 1000])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.7629, 0.4283, 0.3406, 0.3791, 0.3136, 0.2194, 0.2582, 0.3352, 0.5389,
         0.5680

In [13]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F20/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F20-200.npz
Computational Resource: cpu
tensor([[11.3386, 13.1263, 13.8767,  ...,  5.7389,  6.8761,  6.2741],
        [ 5.7211, 14.4593, 14.3088,  ...,  1.3159,  2.4212,  2.8584],
        [ 4.0447,  2.1028, -0.9289,  ..., -1.8900,  0.4822,  2.7293],
        ...,
        [10.7340, 11.0293, 13.2744,  ..., 19.1738, 18.7282,  2.5316],
        [13.4577, 11.8053, 13.2972,  ..., -4.3113, -0.7388, 16.2375],
        [-4.9094, -0.9069,  2.7131,  ..., -5.2295,  4.3459,  3.2145]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.6568, 0.5110, 0.5316, 0.3658, 0.3555, 0.5001, 0.3437, 0.4254, 0.5043,
         0.6429],

In [14]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F20/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-169-F20-500.npz
Computational Resource: cpu
tensor([[-11.8139,  -8.8535,  -9.0403,  ...,   6.4870,   2.9523,  -3.6507],
        [  6.6273,  -2.4155,  -7.0356,  ...,   0.6652,   0.7437,   2.8152],
        [  9.8956,  15.3371,  10.1312,  ...,   0.5819,   3.6557,   7.4309],
        ...,
        [ -2.6283,  -6.1774,  -4.0181,  ...,   0.7161,  -0.9069,   0.9521],
        [  4.1426,   1.7092,   0.2607,  ...,   9.0017,   9.5658,  10.5313],
        [  3.9112,   3.7291,   5.1713,  ...,   2.6358,   9.2110,   9.7277]])
shape: torch.Size([10, 500])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.6345, 0.4363, 0.3985, 0.3521, 0.3063, 0.2807, 0.186

In [12]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/lorenz/F20/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "lorenz",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "10",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: lorenz-381-F20-1000.npz
Computational Resource: cpu
tensor([[  7.9987,   3.9903,   3.6880,  ...,   0.9617,  -8.6353,  -5.2162],
        [-10.2784, -10.2385,  -7.4614,  ...,  26.7369,  24.4717,   0.9279],
        [  2.9866,   5.6446,  -4.7878,  ...,   2.4040,   0.8049,  14.4030],
        ...,
        [  3.2711,  16.1524,  14.1159,  ...,   1.7868,  -0.8768,   0.1724],
        [-15.8695,   7.0099,  13.2193,  ...,  11.0392,  11.6731,  13.1591],
        [  8.9278,   9.2394,  -7.5752,  ...,  -8.0850,  -9.9189, -18.4429]])
shape: torch.Size([10, 1000])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 1]
 [1 1 1 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 1 0 0 0 0]
 [0 0 0 1 1 1 1 0 0 0]
 [0 0 0 0 1 1 1 1 0 0]
 [0 0 0 0 0 1 1 1 1 0]
 [0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 1 1 1]]
tensor([[0.6554, 0.4240, 0.4291, 0.2981, 0.2453, 0.3421, 0.2

In [1]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag2-sp2/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var2-594-2-0-200.npz
Computational Resource: cpu
tensor([[ 0.0961,  0.0965,  0.0774,  ...,  0.0304, -0.0037, -0.1426],
        [ 0.1101,  0.1096,  0.2828,  ..., -0.0235, -0.1462, -0.1887],
        [ 0.2602,  0.2592,  0.1814,  ..., -0.1545, -0.1137, -0.1374],
        ...,
        [-0.0803, -0.0248, -0.0796,  ..., -0.1332, -0.1401, -0.0763],
        [ 0.0136, -0.0680, -0.0618,  ..., -0.1702, -0.1599, -0.0877],
        [ 0.0208,  0.0618, -0.0321,  ...,  0.2440,  0.2361,  0.4746]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 0 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 1 1 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1]]
tensor([[0.6301, 0.4825, 0.3905, 0.4624, 0.3959, 0.2841, 0.3365, 0.4240, 0.4479,
         0.4250],
 

In [2]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp2/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var3-381-2-0-200.npz
Computational Resource: cpu
tensor([[ 0.1039, -0.0137, -0.0515,  ..., -0.0468,  0.0052, -0.1270],
        [-0.1137,  0.0369,  0.0678,  ..., -0.1842,  0.1090, -0.0466],
        [ 0.0063,  0.1662,  0.0598,  ...,  0.0620, -0.0307,  0.1054],
        ...,
        [ 0.1568,  0.0722,  0.0338,  ..., -0.1096, -0.0805, -0.0471],
        [ 0.1597,  0.0757,  0.1237,  ...,  0.2032,  0.1473,  0.1046],
        [-0.0753, -0.1219, -0.0317,  ...,  0.0299, -0.0632, -0.1342]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 0 0 0 0 0 0 0 1 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1]]
tensor([[0.6203, 0.3509, 0.4130, 0.4390, 0.4197, 0.2838, 0.3129, 0.5046, 0.5435,
         0.4182],
 

In [3]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp3/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var3-381-3-0-200.npz
Computational Resource: cpu
tensor([[ 0.0311, -0.0977, -0.1210,  ..., -0.0674, -0.0351, -0.1392],
        [-0.1343,  0.0333,  0.0399,  ..., -0.1133,  0.1711,  0.0043],
        [-0.0617,  0.1009, -0.0308,  ...,  0.1153, -0.0093,  0.1112],
        ...,
        [ 0.0370, -0.0359, -0.0725,  ..., -0.0877, -0.0541, -0.0149],
        [ 0.1473,  0.0429,  0.0759,  ...,  0.1876,  0.1323,  0.0834],
        [-0.1703, -0.2086, -0.0928,  ..., -0.0177, -0.0761, -0.1347]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 1 0 0 0 0 0 0 1 0]
 [0 1 1 1 0 0 0 0 0 0]
 [0 0 1 1 0 1 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 1]
 [0 0 0 0 1 1 0 0 1 0]
 [1 1 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 1]
 [0 0 0 1 0 0 1 1 0 0]
 [0 0 0 1 0 0 1 0 1 0]
 [0 0 1 0 0 0 1 0 0 1]]
tensor([[0.6285, 0.3673, 0.3719, 0.4168, 0.4357, 0.2611, 0.3119, 0.5152, 0.5258,
         0.4607],
 

In [4]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp4/time200"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "200",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var3-618-4-0-200.npz
Computational Resource: cpu
tensor([[ 0.1519,  0.1336,  0.0118,  ..., -0.0461,  0.0480, -0.1797],
        [ 0.0234, -0.0030,  0.1654,  ..., -0.1433,  0.0269, -0.0524],
        [-0.0193,  0.0182,  0.0443,  ...,  0.1470, -0.0853, -0.0781],
        ...,
        [ 0.0321, -0.0058,  0.1165,  ...,  0.0165, -0.0723,  0.0213],
        [ 0.0380,  0.1569, -0.0706,  ..., -0.1726, -0.0157, -0.0588],
        [ 0.1481,  0.0552,  0.1027,  ..., -0.0646,  0.0524,  0.0924]])
shape: torch.Size([10, 200])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 0 0 0 1 0 1 1 0 0]
 [1 1 0 0 0 1 0 0 0 1]
 [0 0 1 1 1 0 0 0 1 0]
 [0 0 0 1 0 1 1 0 0 1]
 [0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 1 1 1 0 0 1]
 [0 0 1 0 0 0 1 0 1 1]
 [0 1 1 0 0 1 0 1 0 0]
 [0 0 0 0 1 1 0 1 1 0]
 [1 0 0 1 0 1 0 0 0 1]]
tensor([[0.5734, 0.3516, 0.3314, 0.4878, 0.4177, 0.2805, 0.3400, 0.5132, 0.4618,
         0.4154],
 

In [5]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp2/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var3-381-2-0-500.npz
Computational Resource: cpu
tensor([[ 0.1175,  0.0126,  0.0023,  ..., -0.0339,  0.1847,  0.0385],
        [ 0.0469,  0.1950,  0.0884,  ...,  0.1051,  0.1031, -0.0453],
        [ 0.1180,  0.0591,  0.1284,  ..., -0.0919, -0.0041, -0.0416],
        ...,
        [-0.0571, -0.1282, -0.0444,  ...,  0.2967,  0.0534,  0.0720],
        [ 0.1548,  0.2127,  0.1431,  ...,  0.0251,  0.1831,  0.0202],
        [ 0.0212, -0.0779,  0.0036,  ...,  0.0931,  0.0251,  0.1366]])
shape: torch.Size([10, 500])
node = 0
1041
node = 1
1041
node = 2
1041
node = 3
1041
node = 4
1041
node = 5
1041
node = 6
1041
node = 7
1041
node = 8
1041
node = 9
1041
[[1 0 0 0 0 0 0 0 1 0]
 [0 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 1 1 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 1]]
tensor([[0.6529, 0.3578, 0.3725, 0.4447, 0.4090, 0.2868, 0.3121, 0.3860, 0.4788,
         0.4113],
 

In [6]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag2-sp2/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


Processing file: var2-169-2-0-500.npz
Computational Resource: cpu
tensor([[-0.2471, -0.2290, -0.2777,  ..., -0.0557, -0.1138,  0.0744],
        [-0.0811,  0.0562, -0.1847,  ...,  0.0575,  0.0178,  0.0248],
        [ 0.0120, -0.2236, -0.2399,  ..., -0.1569, -0.0527,  0.0326],
        ...,
        [ 0.0050, -0.2497,  0.0337,  ...,  0.0305,  0.1566, -0.1010],
        [-0.2581, -0.0713, -0.1694,  ...,  0.1928,  0.1513,  0.0262],
        [-0.0067, -0.1392, -0.0864,  ..., -0.0638,  0.0868,  0.0690]])
shape: torch.Size([10, 500])
node = 0
1041


Traceback (most recent call last):
  File "main2.py", line 296, in <module>
    trainVerboseLvl)
  File "/root/autodl-tmp/sru/models/sru.py", line 191, in trainSRU
    sum([smooth_loss_list[i] for i in range(blk_size - 1)]).backward()
  File "/root/miniconda3/envs/sru/lib/python3.7/site-packages/torch/tensor.py", line 102, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/root/miniconda3/envs/sru/lib/python3.7/site-packages/torch/autograd/__init__.py", line 90, in backward
    allow_unreachable=True)  # allow_unreachable flag
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp3/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp4/time500"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "500",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag2-sp2/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp2/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp3/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [ ]:
import os
import subprocess

import numpy as np
import torch
from sklearn.metrics import roc_auc_score

# 设定路径
# dataset_dir = "my_dataset/lorenz96"  # 存储lorenz数据的文件夹
dataset_dir = "datasets/var/lag3-sp4/time1000"  # 存储lorenz数据的文件夹
output_dir = "output/logs"  # 存储输出日志的目录

# 获取所有的npz文件
npz_files = [f for f in os.listdir(dataset_dir) if f.endswith('.npz')]

# 准备AUROC结果存储
auroc_results = []

# 遍历每个文件，执行main.py，计算AUROC
for npz_file in npz_files:
    print(f"Processing file: {npz_file}")

    # 数据文件路径
    npz_file_path = dataset_dir + "/" + npz_file

    # 构建命令行参数
    command = [
        "python", "main2.py",
        "--dataset", "var",
        "--dsid", "1",  # 这里使用1，假设每个数据集有一个dsid
        "--model", "sru",
        "--sruname", "10",  # 设置为10，按照要求
        "--T", "1000",  # 设置时间步数
        "--F", "30",  # 设置F
        "--nepochs", "2000",  # 训练2000个epoch
        "--mu1", "0.021544",  # mu1的值
        "--mu2", "0.031623",  # mu2的值
        "--mu3", "0.464159",  # mu3的值
        "--lr", "0.001",  # 学习率
        "--joblog", output_dir + "/" + f"{npz_file}_output.npz",  # 输出路径
        "--inputfile", npz_file_path  # 传递数据文件路径
    ]

    # 执行命令
    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error processing {npz_file}: {e}")
        continue

    # 输出文件路径（假设每个文件的结果都保存在对应joblog输出路径）
    output_file_path = output_dir + "/" + f"{npz_file}_output.npz"

    # 读取输出的结果文件（假设它包含了Gest和Gref）
    try:
        result = np.load(output_file_path)
        Gest = torch.tensor(result["Gest"])  # 预测值
        Gref = torch.tensor(result["Gref"])  # 真实值

        # 计算 AUROC
        # 假设Gest和Gref都是每个节点的预测和真实值
        # 如果你有多个节点，计算每个节点的AUROC并取平均
        auroc_values = []
        for i in range(Gest.shape[0]):  # 遍历每个节点
            try:
                auroc = roc_auc_score(Gref[i].cpu().numpy(), Gest[i].cpu().numpy())
                auroc_values.append(auroc)
            except ValueError as ve:
                print(f"Error computing AUROC for node {i}: {ve}")
                auroc_values.append(np.nan)

        # 计算该文件的平均AUROC值
        avg_auroc = np.nanmean(auroc_values)
        auroc_results.append((npz_file, avg_auroc))
        print(f"AUROC for {npz_file}: {avg_auroc:.4f}")

    except Exception as e:
        print(f"Error reading output file for {npz_file}: {e}")
        continue

# 输出所有文件的AUROC结果
print("\nAll AUROC Results:")
for file, auroc in auroc_results:
    print(f"{file}: {auroc:.4f}")


In [10]:
import torch

print(torch.cuda.is_available())  # 检查是否能够使用 GPU
print(torch.version.cuda)  # 查看当前 CUDA 版本
print(torch.cuda.current_device())  # 查看当前设备的 ID


True
10.0.130
0


In [6]:
torch.cuda.empty_cache()
